# 🎬 YouTube 视频转录工具

将 YouTube 视频转换为文字和字幕文件 (txt + srt)

**使用步骤：**
1. 📁 运行「挂载 Google Drive」
2. 📦 运行「安装依赖」
3. ⚙️ 填写配置（YouTube URL、模型等）
4. 🚀 运行「执行转录」

> ⚠️ 如遇 403 错误，请查看底部「常见问题」部分

In [ ]:
#@title 📁 挂载 Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 设置输出目录
import os
OUTPUT_DIR = "/content/drive/MyDrive/YouTubeTranscripts"
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"✅ 输出目录: {OUTPUT_DIR}")

# 检查 cookies 文件
COOKIES_PATH = "/content/drive/MyDrive/cookies.txt"
if os.path.exists(COOKIES_PATH):
    print(f"🍪 发现 cookies 文件: {COOKIES_PATH}")
else:
    print("ℹ️ 未发现 cookies.txt，如遇 403 错误请参考说明上传")

In [ ]:
#@title 📦 安装依赖 (自动检查，已安装则跳过)

import subprocess
import sys
import os

def check_and_install(package_name, import_name=None):
    """检查包是否已安装，未安装则安装"""
    import_name = import_name or package_name
    try:
        __import__(import_name)
        print(f"✅ {package_name} 已安装")
        return True
    except ImportError:
        print(f"📦 正在安装 {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package_name])
        print(f"✅ {package_name} 安装完成")
        return False

# 安装 Deno JavaScript 运行时 (yt-dlp 2025+ 必需)
print("📦 检查/安装 Deno JavaScript 运行时...")
deno_path = os.path.expanduser("~/.deno/bin/deno")
if not os.path.exists(deno_path):
    subprocess.run("curl -fsSL https://deno.land/install.sh | sh", shell=True, check=True)
    print("✅ Deno 安装完成")
else:
    print("✅ Deno 已安装")

# 添加 Deno 到 PATH
os.environ["PATH"] = os.path.expanduser("~/.deno/bin") + os.pathsep + os.environ.get("PATH", "")

# 强制更新 yt-dlp 到最新版本 (解决 403 问题的关键)
print("📦 更新 yt-dlp 到最新版本...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "yt-dlp"])
print("✅ yt-dlp 已更新")

# 检查并安装 whisper
check_and_install("openai-whisper", "whisper")

print("\n✅ 所有依赖就绪！")

In [ ]:
#@title ⚙️ 配置

#@markdown ### YouTube 视频 URL
#@markdown 多个 URL 用 **竖线 `|`** 分隔，例如: `URL1 | URL2 | URL3`
YOUTUBE_URLS = "https://youtu.be/xxxxxxxxxx" #@param {type:"string"}

#@markdown ### 可选设置
MODEL = "large-v3" #@param ["tiny", "base", "small", "medium", "large", "large-v2", "large-v3"]
LANGUAGE = "" #@param {type:"string"}
#@markdown 留空自动检测语言，或指定语言代码如: `zh`, `en`, `ja`

#@markdown ### 403 错误修复
USE_COOKIES = False #@param {type:"boolean"}
#@markdown 启用后将使用 Google Drive 根目录的 cookies.txt

# 解析 URL 列表（支持竖线 | 分隔）
import re
url_list = [u.strip() for u in YOUTUBE_URLS.split('|') if u.strip()]

print(f"🎬 待处理视频数量: {len(url_list)}")
for i, url in enumerate(url_list, 1):
    print(f"   {i}. {url}")
print(f"🤖 模型: {MODEL}")
print(f"🌐 语言: {LANGUAGE if LANGUAGE else '自动检测'}")
print(f"🍪 使用 Cookies: {'是' if USE_COOKIES else '否'}")

In [ ]:
#@title 🚀 执行转录（支持批量）

import re
import os
import sys
import shutil
import tempfile
import time
import importlib.util
from pathlib import Path
import yt_dlp
import whisper

# ===== 工具函数 =====

def parse_youtube_url(url: str) -> str | None:
    """解析 YouTube URL，提取 video_id"""
    patterns = [
        r'(?:https?://)?(?:www\.)?youtube\.com/watch\?v=([a-zA-Z0-9_-]{11})',
        r'(?:https?://)?(?:www\.)?youtube\.com/embed/([a-zA-Z0-9_-]{11})',
        r'(?:https?://)?(?:www\.)?youtube\.com/v/([a-zA-Z0-9_-]{11})',
        r'(?:https?://)?youtu\.be/([a-zA-Z0-9_-]{11})',
    ]
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    return None


def sanitize_filename(filename: str, max_bytes: int = 180) -> str:
    """清理文件名，移除非法字符，并限制字节长度（兼容中文）"""
    illegal_chars = r'[<>:"/\\|?*]'
    sanitized = re.sub(illegal_chars, '_', filename)
    # 按字节长度截断（UTF-8 中文占 3 字节，留余量给扩展名）
    encoded = sanitized.encode('utf-8')
    if len(encoded) > max_bytes:
        result = ''
        current_bytes = 0
        for char in sanitized:
            char_bytes = len(char.encode('utf-8'))
            if current_bytes + char_bytes > max_bytes:
                break
            result += char
            current_bytes += char_bytes
        return result
    return sanitized


def format_timestamp(seconds: float) -> str:
    """将秒数转换为 SRT 时间戳格式 (HH:MM:SS,mmm)"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millis = int((seconds % 1) * 1000)
    return f"{hours:02d}:{minutes:02d}:{secs:02d},{millis:03d}"


def format_duration(seconds: float) -> str:
    """将秒数格式化为可读时长"""
    if seconds < 60:
        return f"{seconds:.1f}秒"
    elif seconds < 3600:
        mins = int(seconds // 60)
        secs = int(seconds % 60)
        return f"{mins}分{secs}秒"
    else:
        hours = int(seconds // 3600)
        mins = int((seconds % 3600) // 60)
        return f"{hours}小时{mins}分"


def detect_js_runtime() -> dict | None:
    """自动检测可用 JS runtime"""
    runtime_bins = {
        "deno": ["deno"],
        "bun": ["bun"],
        "node": ["node"],
        "quickjs": ["qjs", "quickjs"],
    }
    for name, bins in runtime_bins.items():
        for bin_name in bins:
            path = shutil.which(bin_name)
            if path:
                return {name: {"path": path}}
    return None


def find_cookie_file(search_dir: Path) -> Path | None:
    """在指定目录中查找 cookies 文件"""
    if not search_dir.exists():
        return None
    preferred = search_dir / "cookies.txt"
    if preferred.exists():
        return preferred
    candidates = list(search_dir.glob("*.txt"))
    if not candidates:
        return None
    candidates.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    return candidates[0]


def preflight_check(cookies_path: str | None, js_runtimes: dict | None) -> None:
    """执行环境自检并输出提示"""
    print("🔍 环境自检...")

    # yt-dlp 版本
    try:
        from yt_dlp import version as ydl_version
        print(f"   yt-dlp: {ydl_version.__version__}")
    except Exception:
        print("   yt-dlp: unknown")

    # JS runtime
    if js_runtimes:
        print(f"   JS runtime: {', '.join(js_runtimes.keys())}")
    else:
        print("   JS runtime: 未检测到")
        print("     ⚠️ 建议安装 Deno (运行上方的安装依赖单元格)")

    # EJS 组件检测
    ejs_local = False
    if importlib.util.find_spec("yt_dlp_ejs") is not None:
        ejs_local = True
    elif importlib.util.find_spec("ytdlp_ejs") is not None:
        ejs_local = True

    if ejs_local:
        print("   EJS: 已安装 (yt-dlp-ejs)")
    else:
        print("   EJS: 未检测到 (部分视频可能需要)")

    # Cookies 来源
    if cookies_path and Path(cookies_path).exists():
        print(f"   Cookies: {cookies_path}")
    else:
        print("   Cookies: 未配置")


# ===== 核心功能 =====

def download_audio(
    url: str,
    output_dir: Path,
    cookies_path: str | None = None,
    js_runtimes: dict | None = None
) -> tuple[Path, str]:
    """使用 yt-dlp 下载音频，尝试多种客户端绕过 403"""
    print("📥 正在下载音频...")

    cookies_opts: dict = {}
    if cookies_path and Path(cookies_path).exists():
        cookies_opts['cookiefile'] = cookies_path
        print(f"🍪 使用 cookies: {cookies_path}")

    # 基础配置
    base_opts = {'quiet': True}
    if js_runtimes:
        base_opts['js_runtimes'] = js_runtimes
    base_opts.update(cookies_opts)

    # 获取视频信息
    with yt_dlp.YoutubeDL(base_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        video_title = info.get('title', 'unknown')

    safe_title = sanitize_filename(video_title)
    output_template = str(output_dir / f"{safe_title}.%(ext)s")

    # 尝试多种客户端配置来绕过 403
    client_configs = [
        {'player_client': ['ios', 'web']},
        {'player_client': ['android', 'web']},
        {'player_client': ['tv', 'web']},
        {},
    ]

    last_error = None
    for extractor_args in client_configs:
        client_name = extractor_args.get('player_client', 'default')
        print(f"🔄 尝试客户端: {client_name}")

        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': output_template,
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'quiet': False,
            'no_warnings': False,
        }
        if js_runtimes:
            ydl_opts['js_runtimes'] = js_runtimes
        ydl_opts.update(cookies_opts)
        if extractor_args:
            ydl_opts['extractor_args'] = {'youtube': extractor_args}

        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([url])

            audio_path = output_dir / f"{safe_title}.mp3"
            if audio_path.exists():
                print(f"✅ 音频下载完成: {audio_path.name}")
                return audio_path, video_title
        except Exception as e:
            last_error = e
            print(f"⚠️ 客户端 {client_name} 失败: {str(e)[:120]}")

    raise Exception(f"❌ 所有下载方式都失败。最后错误: {last_error}\n\n"
                    "💡 建议：请启用 Cookies 功能（参考底部说明上传 cookies.txt）")


def transcribe_audio(audio_path: Path, model_name: str, language: str = None) -> dict:
    """使用 Whisper 转录音频"""
    print(f"🔄 正在加载 Whisper 模型 ({model_name})...")
    model = whisper.load_model(model_name)

    # 获取音频时长
    try:
        from whisper.audio import load_audio, SAMPLE_RATE
        audio = load_audio(str(audio_path))
        audio_duration = len(audio) / SAMPLE_RATE
    except Exception:
        audio_duration = 0.0

    if audio_duration > 0:
        print(f"🎵 音频时长: {format_duration(audio_duration)}")

    print("🎙️ 正在转录音频...")

    transcribe_options = {'fp16': True, 'verbose': True}
    if language:
        transcribe_options['language'] = language

    start_time = time.time()
    result = model.transcribe(str(audio_path), **transcribe_options)
    elapsed_time = time.time() - start_time

    detected_lang = result.get('language', 'unknown')
    print(f"\n✅ 转录完成！检测到语言: {detected_lang}")
    print(f"⏱️  处理耗时: {format_duration(elapsed_time)}")

    if audio_duration > 0:
        speed_ratio = audio_duration / elapsed_time
        print(f"🚀 处理速度: {speed_ratio:.2f}x 实时速度")

    return result


def save_outputs(result: dict, output_dir: str, title: str) -> tuple[Path, Path]:
    """保存 txt 和 srt 文件"""
    safe_title = sanitize_filename(title)
    output_path = Path(output_dir)

    # 保存 txt
    txt_file = output_path / f"{safe_title}.txt"
    with open(txt_file, 'w', encoding='utf-8') as f:
        f.write(result['text'].strip())
    print(f"📄 文本文件已保存: {txt_file}")

    # 保存 srt
    srt_file = output_path / f"{safe_title}.srt"
    segments = result.get('segments', [])
    with open(srt_file, 'w', encoding='utf-8') as f:
        for i, segment in enumerate(segments, start=1):
            start_time = format_timestamp(segment['start'])
            end_time = format_timestamp(segment['end'])
            text = segment['text'].strip()
            f.write(f"{i}\n{start_time} --> {end_time}\n{text}\n\n")
    print(f"📄 字幕文件已保存: {srt_file}")

    return txt_file, srt_file


def process_single_video(
    url: str,
    output_dir: str,
    model: str,
    language: str | None,
    cookies_path: str | None,
    js_runtimes: dict | None
) -> bool:
    """处理单个视频，返回是否成功"""
    video_id = parse_youtube_url(url)
    if not video_id:
        print(f"❌ 无效的 YouTube URL: {url}")
        return False

    print(f"🎬 Video ID: {video_id}")

    # 创建临时目录
    temp_dir = Path(tempfile.mkdtemp())

    try:
        # 下载音频
        audio_path, video_title = download_audio(
            url,
            temp_dir,
            cookies_path=cookies_path,
            js_runtimes=js_runtimes
        )

        # 转录
        result = transcribe_audio(audio_path, model, language)

        # 保存
        txt_file, srt_file = save_outputs(result, output_dir, video_title)

        print(f"✅ 完成: {txt_file.name}")
        return True

    except Exception as e:
        print(f"❌ 处理失败: {e}")
        return False

    finally:
        # 清理临时文件
        if temp_dir.exists():
            shutil.rmtree(temp_dir, ignore_errors=True)


def disconnect_runtime():
    """断开 Colab 运行时以节省流量"""
    try:
        from google.colab import runtime
        print("🔌 3 秒后断开运行时...")
        time.sleep(3)
        runtime.unassign()
    except Exception:
        # 非 Colab 环境则忽略
        pass


# ===== 主执行流程 =====

if not url_list:
    raise ValueError("❌ 未提供任何 URL，请在配置单元格中填写")

print(f"📋 批量任务: 共 {len(url_list)} 个视频")
print("=" * 50)

# 检测 JS runtime
js_runtimes = detect_js_runtime()

# 设置 cookies 路径
cookies_path = COOKIES_PATH if USE_COOKIES else None

# 环境自检
preflight_check(cookies_path, js_runtimes)
print("=" * 50)

# 批量处理
success_count = 0
fail_count = 0
failed_urls = []

for idx, url in enumerate(url_list, 1):
    print(f"\n{'='*50}")
    print(f"📌 [{idx}/{len(url_list)}] 处理中...")
    print(f"🔗 {url}")
    print("-" * 50)

    language_opt = LANGUAGE if LANGUAGE else None
    success = process_single_video(
        url,
        OUTPUT_DIR,
        MODEL,
        language_opt,
        cookies_path,
        js_runtimes
    )

    if success:
        success_count += 1
    else:
        fail_count += 1
        failed_urls.append(url)

# 汇总报告
print("\n" + "=" * 50)
print("📊 批量处理完成！")
print(f"   ✅ 成功: {success_count}")
print(f"   ❌ 失败: {fail_count}")
print(f"📁 文件位置: {OUTPUT_DIR}")

if failed_urls:
    print("\n⚠️ 失败的 URL:")
    for url in failed_urls:
        print(f"   - {url}")

print("🧹 临时文件已清理")

# 断开运行时节省流量
disconnect_runtime()

---

## 📋 说明

### Whisper 模型选择
| 模型 | 大小 | 速度 | 准确度 |
|------|------|------|--------|
| tiny | 39M | 最快 | 较低 |
| base | 74M | 快 | 一般 |
| small | 244M | 中等 | 良好 |
| medium | 769M | 较慢 | 很好 |
| large-v3 | 1550M | 慢 | 最佳 |

### 输出文件
- `.txt` - 纯文本转录内容
- `.srt` - 带时间轴的字幕文件，可导入视频播放器

### 文件位置
输出文件保存在: `Google Drive/YouTubeTranscripts/`

---

## 🔧 常见问题

### HTTP 403 Forbidden 错误
这是 YouTube 的反爬虫机制，解决方法：

**方法 1: 使用 Cookies (推荐)**
1. 安装浏览器扩展 [Get cookies.txt LOCALLY](https://chromewebstore.google.com/detail/get-cookiestxt-locally/cclelndahbckbenkjhflpdbgdldlbecc)
2. 登录 YouTube 账号
3. 在 YouTube 页面点击扩展，导出 cookies.txt
4. 上传到 Google Drive 根目录 (MyDrive/cookies.txt)
5. 在配置中设置 `USE_COOKIES = True`

**方法 2: 等待 yt-dlp 更新**
- yt-dlp 会定期更新以适应 YouTube 变化
- 重新运行「安装依赖」单元格会自动更新到最新版

### 视频无法下载
以下视频可能无法下载：
- 私享视频 (Private)
- 年龄限制视频 (需登录 + Cookies)
- 地区限制视频
- 付费/会员专享内容